In [ ]:
# from langchain.document_loaders import PyMuPDFLoader

# def load_file_law(filename):
#     doc_file = f"data/tax_law/{filename}.doc"
# load_file_law('개별소비세법')

## 세법 챗봇
#### 데이터 수집: 개별소비세법
- 개별소비세법.pdf
- 개별소비세법_시행규칙.pdf
- 개별소비세법_시행령.pdf

### 패키지 설치

In [1]:
%pip install -q langchain langchain-huggingface langchain-community langchain-core langchain-text-splitters bitsandbytes docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -U langchain-community pymupdf langchain_chroma langchain_huggingface

### 문서 전처리 및 split

In [1]:
def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [20]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.schema import Document
import re
def load_law(filename):
    
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.
    """

    file_path = f"data/tax_law/{filename}.pdf"

    loader = PyMuPDFLoader(file_path)
    load_document = loader.load()

    # 전처리 - 반복 텍스트 삭제
    delete_patterns = [
        rf"법제처\s*\d+\s*국가법령정보센터\n{filename.replace('_', ' ')}\n",
        r'\[[\s\S]*?\]', 
        r'<[\s\S]*?>',
    ]
    
    full_text = " ".join([document.page_content for document in load_document])
    
    for pattern in delete_patterns:
        full_text = re.sub(pattern, "", full_text)
        full_text = full_text.replace("(", " (")
    
    # 전처리 - split
    split_pattern = r"\s*\n(제\d+조(?:의\d+)?(?:\([^)]*\))?)(?=\s|$)"
    chunks = re.split(split_pattern, full_text)

    chunk_docs = []
    connected_chunks = [] 
    current_chunk = ""
    is_buchik_section = False 
    
    # 전처리 - 일반 조항과 부칙의 조항과 구별하기 위해 접두어 '부칙-'을 넣음.
    for chunk in chunks:
        if re.search(r"\n\s*부칙", chunk): 
            is_buchik_section = True

        if chunk.startswith("제") and "조" in chunk: 
            if is_buchik_section:
                chunk = "부칙-" + chunk

            if current_chunk:
                connected_chunks.append(current_chunk.strip())
            current_chunk = chunk 
        else:
            current_chunk += f" {chunk}" 

    if current_chunk:
        connected_chunks.append(current_chunk.strip())

    for chunk in connected_chunks:
        pattern =  r"^(?:부칙-)?제\d+조(?:의\d*)?(?:\([^)]*\))?"

        keyword = ''
        keyword += f"{filename.replace("_", " ")} "
        
        match = re.search(pattern, chunk)
        word=''
        if match:
            word = match.group()
            word = re.sub(r'\s+', ' ', word)
            word = re.sub(r'\(+', ' (', word)
        else:
            word = "관련 부서 연락처"
        
        keyword += word 

        parts = filename.split("_")

        # 기본적으로 두 부분이 있을 것으로 예상, 없을 경우 '법'로 기본값 설정
        if len(parts) == 2:
            law_name, doc_type = parts
        else:
            law_name = parts[0]
            doc_type = "법률"  # 기본값 설정

        
        doc = Document(
            metadata={
                "document_type": doc_type,
                "law_name": law_name,
                "article_number": f'{doc_type} {word}',
                "description": f"{keyword}에 관한 문서입니다.",
                "source": f'{filename}.pdf', 
            }, 
            page_content=chunk
        ),
        
        chunk_docs.extend(doc)
        
    return chunk_docs

In [111]:
# document_list = []

# law_files = get_file_names("data/tax_law")
# for file in law_files:
#     document_list.extend(load_law(file))

In [21]:
document_list = []
document_list.extend(load_law('개별소비세법'))
document_list.extend(load_law('개별소비세법_시행령'))
document_list.extend(load_law('개별소비세법_시행규칙'))
len(document_list)

132

### Chroma를 활용한 vector store 구성

In [ ]:
# from langchain_chroma import Chroma
# from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large-instruct')

# COLLECTION_NAME = "law"
# PERSIST_DIRECTORY = "vector_store_hf"

# vector_store_hf = Chroma.from_documents(
#     documents=document_list,
#     embedding=embeddings,
#     collection_name=COLLECTION_NAME,
#     persist_directory=PERSIST_DIRECTORY
# )


In [23]:
# 📌 vector store 생성
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

COLLECTION_NAME = "law_5"
PERSIST_DIRECTORY = "vector_store9"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    
vector_store = set_vector_store(document_list)

### Chain 만들기 
- LLM
- Prompt Template
- Retriever

In [36]:
from langchain_openai import  ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [37]:
from langchain.prompts import ChatPromptTemplate
# Prompt Template 생성
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 

    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
   

    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)

In [38]:
retriever = vector_store.as_retriever(
    search_kwargs={"k":3}
)

In [39]:
retriever.invoke("개별소비세법 제1조")

[Document(metadata={'article_number': '시행규칙 관련 부서 연락처', 'description': '개별소비세법 시행규칙 관련 부서 연락처에 관한 문서입니다.', 'document_type': '시행규칙', 'law_name': '개별소비세법', 'source': '개별소비세법_시행규칙.pdf'}, page_content='개별소비세법 시행규칙\n \n기획재정부    (환경에너지세제과) 044-215-4331, 4336\n기획재정부    (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'article_number': '시행령 제1조', 'description': '개별소비세법 시행령 제1조에 관한 문서입니다.', 'document_type': '시행령', 'law_name': '개별소비세법', 'source': '개별소비세법_시행령.pdf'}, page_content='제1조    (과세물품ㆍ과세장소 및 과세유흥장소의 세목등) 「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n   (「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.'),
 Document(metadata={'article_number': '법률 제4조', 'description': '개별소비세법 제4조에 관한 문서입니다.', 'document_type': '법률', 'law_name': '개별소비세법', 'source': '개별소비세법.pdf'}, page_content='제4조    (과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

parser = StrOutputParser()

chain = {"context": retriever, "question": RunnablePassthrough() } | prompt_template | model | parser

In [34]:
result = chain.invoke("개별소비세법 제1조에 대해 알려줘")

In [13]:
print(result)

개별소비세법 제1조는 다음과 같은 내용을 포함하고 있습니다:

### 개별소비세법 제1조 (목적)
이 법은 개별소비세의 부과 및 징수에 관한 사항을 규정함으로써, 소비에 대한 공평한 세 부담을 도모하고, 국가 재정의 안정적 운영에 기여하는 것을 목적으로 합니다.

#### 관련 내용 요약:
- 개별소비세는 특정 소비재에 대해 부과되는 세금입니다.
- 이 법은 소비에 따른 세 부담의 형평성을 유지하고, 국가 재정에 기여하기 위해 제정되었습니다.

이 조항은 개별소비세법의 전반적인 목적과 방향성을 제시하고 있습니다.

추가로 궁금한 점이 있으시면, 어떤 부분에 대해 더 알고 싶으신지 말씀해 주세요!


## 데이터 전처리